# Neural Network

Le **Artificial Neural Network**, (_NN_) sono modelli matematici largamente utilizzati nel campo dell'**Intelligenza Artificiale** (_AI_) che permettono, a sistemi automatici, di compiere task complessi e articolati che dei semplici algoritmi (es. algoritmi sequenziali) non sarebbero in grado di portare a termine in modo rapido ed efficace.

Le basi di questo metodo risalgono alla metà del XX secolo quando per la prima volta furono proposti algoritmi per l'apprendimento automatico. L'obiettivo era quello di creare _strutture_ in grado di modellare un determinato fenomeno e riproporne il comportamento in determinate condizioni.

Il componente base di questa nuova struttura è il _**neurone**_. Con questo termine identifichiamo un nodo in grado di simulare il comportamento di un neurone biologico e di interconnettersi con altri neuroni al fine di creare una rete. Ogni nodo elabora i segnali ricevuti e trasmette il risultato a nodi successivi.

Un tipico esempio di struttura base di queste reti è il **percettrone**:
![Percettrone](./img/percetrone.png)

Ogni singolo ingresso di questi nodi riceve informazioni che vengono elaborate. L'elaborazione, che in base agli ingressi può diventare complessa, si può pensare come singoli ingressi che vengono moltiplicati per un opportuno valore detto peso. Il risultato ottenuto delle moltiplicazioni viene sommato e se la somma supera una certa soglia il neurone attiva la sua uscita. Il peso serve a quantificare l'importanza di una interconnessione, infatti un ingresso molto importante avrà un peso elevato, mentre un ingresso poco utile all'elaborazione avrà un peso inferiore.

Ponendo in cascata e combinando tra loro più più neuroni generiamo quella che definiamo _NN_.
![ANN](./img/ann.png)

L'utilizzo delle NN è tornato in uso dopo la reinvenzione dell’algoritmo di apprendimento chiamato back-propagation. Questo algoritmo infatti permette di modificare i pesi delle interconnessioni in modo tale che si minimizzi una certa funzione errore E.

Con l'avvento di nuove metodologie come il **machine learning** e l'aumento della performance delle NN il campo dell'intelligenza artificiale è diventato tra i più importanti ambiti di ricerca nella computer scienze. Grazie a questo, i risultati e i campi applicativi acquisiscono, di giorno in giorno, maggiore interesse. Siamo così passati ad analizzare, attraverso le _NN_, problemi sempre più complessi: è l'avvento del **deep learning**.

Nelle reti neurali classiche moderne è possibile riscontrare la presenza di qualche strato nascosto. Questi strati, denominati _hidden layers_, possono essere interpretati come il _cuore_ della rete stessa poiché sono quelli che si occupano di interpretare le features sottomesse alla rete.

![NN](./img/simple_neural_network_header.jpg)

Con il deep learning, invece, siamo rapportati a problemi più complessi: _dal riconoscimento ed interpretazione del linguaggio naturale fino alla visione artificiale_. Nei modelli deep ogni singolo stato nascosto potrebbe essere paragonato ad una piccola rete neurale classica: ponendone in cascata una all'altra possiamo ottenere modelli complessi per la gestione di task anche molto avanzati come ad esempio gli algoritmi di visione per l'_automotive_

![deep](./img/deep.png)

## Caso di studio

### Codice utlizzato per la Neural Network applicato al  nostro caso di studio

Il nostro dataset è composto da 3815 elementi ognuno dei quali è descritto attraverso 31 features. In questo primo approccio viene mostrato come addestrare una rete neurale classica sia attraverso la classica strarificazione per determinare dataset di treaning e test, sia con la cross validation.

In [16]:
# standard libraries
import os
import shutil
import numpy as np
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout
# keras libraries
from keras.models import Sequential
from keras.models import load_model
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
# sklearn libraries
from sklearn.model_selection import train_test_split

# convolutional network parmas
path_dataset = './dataset/dataset_total.txt'  # datasetpath
path_best = './best_model/'  # kfolds model path
path_thebest = './thebetter_model/'  # bset models path

# neural network params
batch_size = 32  # training cases batch
num_epochs = 500  # max number of epochs
num_classes = 3  # number of class in dataset
seed = 42  # base random seed
n_splits = 10  # number of kfold
n_input_layer = 31 # number of inputs layer

Definiamo la funzione utile per il caricamento del dataset e il suo _splitting_ in porzione di test e di training. La strategia adottata è quella della stratificazione: ovvero creare du insiemi nei quali la "_concentrazione_" dei vari esempi sia equipollente. Ad esempio, se nel dataset di training ci sono il 30% dei campioni di tipo 1 il 40% di tipo 2 e 30% dei campioni di tipo 3, le stesse percentuali saranno adottate per il dataset di test.

In [12]:
def load_data_nn():
    """
    generate dataset based on data in dataset folder
    :return: train and test dataset based on stratification strategy
    """
    dataset = np.loadtxt(path_dataset, delimiter='\t')

    y = np.array(np.ceil(dataset[:, -1])).astype(np.str)
    X = np.array(dataset[:, :-1]).astype(np.float32)

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=seed, stratify=y)

    scaler = preprocessing.StandardScaler().fit(x_train)

    X_train = scaler.transform(x_train)
    X_test = scaler.transform(x_test)

    y_train = np.subtract(y_train.reshape((len(y_train), 1)).astype(np.float32), np.asarray(2.0))
    y_test = np.subtract(y_test.reshape((len(y_test), 1)).astype(np.float32), np.asarray(2.0))

    Y_train = np_utils.to_categorical(y_train, num_classes)
    Y_test = np_utils.to_categorical(y_test, num_classes)
    return X_train, X_test, Y_train, Y_test

Definiamo la funzione che instanzierà il modello della nostra rete neurale. La struttura segue i seguenti parametri:
 - primo livello di ingresso con 31 neuroni. Idealmente sarebbero 1 per ciascuna delle features
 - 4 livelli nascoti con lo stesso numero di neuroni e funzione di attivazione _elu_[1](#cite-DBLP:journals-corr-ClevertUH15).
 - 4 livelli di _Dropout_, ciascuno dopo ogni livello nascoto
 - ultimo livello con 3 neuroni e _softmax_ come funzione di attivazione
 - la funzione di perdita da minimizzare è la _binary_crossentropy_ attraverso l'ottimizzazione _adadelta_

In [13]:
def baseline_model():
    """
    Definition of neural network base model
    :return: model
    """
    base_model = Sequential()
    base_model.add(Dense(n_input_layer, activation='elu',  input_shape=(X_train.shape[1],)))
    #hidden
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.1))
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.1))
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.1))
    base_model.add(Dense(31, activation='elu'))
    base_model.add(Dropout(0.3))
    base_model.add(Dense(3, activation='softmax'))
    base_model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
    return base_model

Definiamo una funzione per la valutazione del modello migliore in uscita dal processo di addestramento in cross validation

In [33]:
def evaluete_nn(X_test, Y_test, best_model=None, model=None):
    """
    Evaluate best model after kfold training
    :param X_test: example images to test best model after kfold
    :param Y_test: labels matching truth to example
    :param best_model: index which identify best model after kfold
    :return: evaluation of best model trough dataset test and save it with loss and accuracy metrics
    """

    # load best model and evaluate it with accuracy and loss
    if(model==None):
        print('Load best model and test it ')
        model = load_model(path_best+'checkpoint-%d.h5' %(best_model))
    score = model.evaluate(X_test, Y_test, verbose=0)  # evaluate model
    y_predict = np.asarray(model.predict(X_test, verbose=0))
    Y_predict = np.argmax(y_predict, axis=1)
    y_test = np.argmax(Y_test, axis=1)
    confmatrix = confusion_matrix(y_test, Y_predict)
    print("\nConfusion Matrix :")
    print(confmatrix)
    class_names = ["0", "1", '2']
    print("\nMetrics => ", model.metrics_names, score)
    print('\nClassification Report : ')
    print(classification_report(y_test, Y_predict, target_names=class_names))
    # save model tested with loss and accuracy
    model.save(path_thebest+'model-'+'{:.4f}'.format(score[0])+'-'+'{:.4f}'.format(score[1])+'.h5')

In [19]:
print('STARTING FITTING NEURAL NETWORK')
if os.path.exists(path_best):
    shutil.rmtree(path_best)
os.mkdir(path_best)
if not os.path.exists(path_thebest):
    os.mkdir(path_thebest)

print('loading data .......\n')
X_train, X_test, Y_train, Y_test = load_data_nn()

print('train examples:')
print(len(X_train))
print('test examples:')
print(len(X_test))


STARTING FITTING NEURAL NETWORK
loading data .......

train examples:
2670
test examples:
1145


Addestriamo inizialmente la rete neurale senza sfruttare la cross validation. Imponiamo un numero massimo di iterazioni pari a 500. Poiché, a casusa dello sbilanciamento del dataset, incorreremo sicuramente in overfitting per gli elementi di appartenenti alla classe _low_risk_, imponiamo anche un **EarlyStopping**, ovvero un listner che monitora l'andamento della funzione di perdita che stiamo minimizzando e, se questa non dovesse avare variazioni pari ad un delta di _0.001_ per più di 10 iterazioni, blocchi l'addestramento della rete se necessario.

In [25]:
model = baseline_model()
model.fit(X_train, Y_train, epochs=num_epochs, batch_size=batch_size, verbose=1,
          callbacks=[TensorBoard(log_dir='./nn/tensorboard/'),
                     EarlyStopping(monitor='loss', min_delta=0.001, patience=10, verbose=2, mode='min')])
scores = model.evaluate(X_train, Y_train, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/500
2670/2670 [==============================] - 0s 68us/step - loss: 0.3355 - acc: 0.8767
Epoch 2/500
2670/2670 [==============================] - 0s 57us/step - loss: 0.1546 - acc: 0.9548
Epoch 3/500
2670/2670 [==============================] - 0s 57us/step - loss: 0.1336 - acc: 0.9576
Epoch 4/500
2670/2670 [==============================] - 0s 61us/step - loss: 0.1225 - acc: 0.9598
Epoch 5/500
2670/2670 [==============================] - 0s 61us/step - loss: 0.1224 - acc: 0.9599
Epoch 6/500
2670/2670 [==============================] - 0s 58us/step - loss: 0.1140 - acc: 0.9604
Epoch 7/500
2670/2670 [==============================] - 0s 58us/step - loss: 0.1119 - acc: 0.9576
Epoch 8/500
2670/2670 [==============================] - 0s 58us/step - loss: 0.1076 - acc: 0.9599
Epoch 9/500
2670/2670 [==============================] - 0s 57us/step - loss: 0.1081 - acc: 0.9594
Epoch 10/500
2670/2670 [==============================] - 0s 60us/step - loss: 0.1063 - acc: 0.9605
Epoch 11/

Valutiamo ora le performance del modello addestrato sul dataset di test

In [30]:
evaluete_nn(X_test, Y_test, model=model)

Metrics =>  ['loss', 'acc'] [0.10511936141934457, 0.96593887129204759]

Confusion Matrix :
[[1073    0    5]
 [  18    5    3]
 [  31    0   10]]

Metrics =>  ['loss', 'acc'] [0.10511936141934457, 0.96593887129204759]

Classification Report : 
             precision    recall  f1-score   support

          0       0.96      1.00      0.98      1078
          1       1.00      0.19      0.32        26
          2       0.56      0.24      0.34        41

avg / total       0.94      0.95      0.94      1145



Passando all'utilizzo della cross validation, utilizziamo per la rete la stessa accortezza utilizzata in precedenza con **EarlyStopping**, in più imponiamo che per ogni *k\_fold* venga salvato il miglior modello in un path specifico. Assieme al modello salviamo anche, per ciasun *best\_model*, anche i rispettivi valori di _accurancy_ in un vettore (_**cvscores**_). In questo modo, alla fine dell'iterazione dei vari k_fold, possiamo selezionare il *better\_model* da valutare con il dataset di test.
Il risultato finale fornisce anche un valore di media per l'accuratezza valutata su tutti i modelli risultati migliori per ciascun k_fold.

In [31]:
# generation kfolds to cross validation process
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
cvscores = []
i = 0

# start cross validation
for train, test in kfold.split(X_train, Y_train):
    model = baseline_model()
    model.fit(X_train[train], Y_train[train], epochs=num_epochs, batch_size=batch_size, verbose=0,
              callbacks=[TensorBoard(log_dir='./nn/tensorboard/'),
                         ModelCheckpoint(path_best+'checkpoint-%d.h5' %(i), monitor='acc', verbose=0,
                                         save_best_only=True, mode='max'),
                         EarlyStopping(monitor='loss', min_delta=0.001, patience=10, verbose=2, mode='min')])
    scores = model.evaluate(X_train[test], Y_train[test], verbose=2)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    i += 1
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 00043: early stopping
acc: 96.50%
Epoch 00060: early stopping
acc: 96.88%
Epoch 00053: early stopping
acc: 96.00%
Epoch 00049: early stopping
acc: 96.25%
Epoch 00047: early stopping
acc: 96.88%
Epoch 00032: early stopping
acc: 96.63%
Epoch 00048: early stopping
acc: 94.26%
Epoch 00042: early stopping
acc: 96.75%
Epoch 00053: early stopping
acc: 96.25%
Epoch 00058: early stopping
acc: 97.75%
96.42% (+/- 0.85%)


A questo punto è opportuno valutare il miglior moello tra quelli risultati ottimali per ciascun k_fold.

In [34]:
# evaluate best model based on higher accuracy
vect_max = np.argmax(cvscores)
evaluete_nn(X_test, Y_test, best_model=vect_max)

Load best model and test it 

Confusion Matrix :
[[1078    0    0]
 [  18    7    1]
 [  35    1    5]]

Metrics =>  ['loss', 'acc'] [0.11256823928205206, 0.96797671541896968]

Classification Report : 
             precision    recall  f1-score   support

          0       0.95      1.00      0.98      1078
          1       0.88      0.27      0.41        26
          2       0.83      0.12      0.21        41

avg / total       0.95      0.95      0.94      1145



##### Considerazioni

Analizzando le metriche base di entrambi i casi di addestramento, possiamo notare che gli andamenti sono pressoché uguali, con una precisione del 96%. Il dato che ci consente di percepire che l'addestramento effettuato con la cross validation sia in qualche modo migliore, ci viene fornito dal report dalla cufusion matrix: nel primo caso si nota chiaramente un che il modello soffre di overfitting per la classe 0 e non ha ottimi riscontri per gli elementi di classe 2 (in questo caso ha una precisione al di sotto del 60%). Mentre per la classe 1, nonostante una precisione molto alta, la recall è molto bassa addirirttura sotto il 20%.

Nel caso di addestramento con la tecnica della cross validazione, benché non possiamo dirci del tutto fuori dal fenomeno di overfitting, possiamo notare una migliore precisione per gli elementi di classe 1 e classe 2. In questo caso siamo oltre il valore di 80% percento per la classe 2, e di poco sotto il 90% per la classe 1. Anche in questo caso la recall per queste 2 classi è molto bassa, indice chiaro di una forte tendenza verso l'overfitting.

L'utilizzo della cross validation, per evitare o attenuare il fenomeno di overfitting sul modello, è stato dettato dal voler rendere il modello il più vicino possibile al _mondo reale_. Infatti, avremmo potuto utilizzare tecniche di _**data augumentation**_ per creare esempi _artificiali_ della classe 1 e della classe 2, avremmo però reso il modello fittizio e molto lontano dal caso reale nel quale dovrebbe agire

### Codice utlizzato per la Deep Neural Network applicato al  nostro caso di studio